# データ探索

このノートブックではJ-Quants APIから取得した日本株データの探索的データ分析を行います。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# 親ディレクトリをパスに追加
sys.path.append('..')

from src.data.jquants_client import JQuantsClient
from src.data.data_processor import standardize_code, filter_topix500
from src.utils.config import load_config

# Jupyter Notebookの表示設定
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

## 設定の読み込み

In [ ]:
# 設定ファイルの読み込み
config = load_config('../configs/train_config.yaml')

# データディレクトリの作成
os.makedirs(config['paths']['data_dir'], exist_ok=True)

## J-Quants APIからデータ取得

APIアクセストークンが事前に設定されている必要があります。

In [ ]:
from datetime import datetime, timedelta

# J-Quants APIクライアントの初期化
client = JQuantsClient(
    mail_address=config.get('jquants', {}).get('mail_address'),
    password=config.get('jquants', {}).get('password'),
    refresh_token=config.get('jquants', {}).get('refresh_token')
)

# データ期間の設定
end_dt = datetime.now()
start_dt = end_dt - timedelta(days=365 * config['data']['historical_years'])

# TOPIX500銘柄の取得
topix500_path = os.path.join(config['paths']['data_dir'], 'topix500.txt')
if os.path.exists(topix500_path):
    # すでにファイルが存在する場合は読み込み
    with open(topix500_path, 'r') as f:
        topix500 = f.read().splitlines()
else:
    # 新規取得
    topix500 = client.get_topix500_tickers(save_path=topix500_path)

print(f"TOPIX500銘柄数: {len(topix500)}")
print(f"サンプル銘柄: {topix500[:10]}")

In [ ]:
# 株価データの取得
stock_price_path = os.path.join(config['paths']['data_dir'], 'stock_price.csv.gz')
if os.path.exists(stock_price_path):
    # すでにファイルが存在する場合は読み込み
    stock_price = pd.read_csv(stock_price_path, compression='gzip')
else:
    # 新規取得
    stock_price = client.get_stock_data(start_dt, end_dt, save_path=stock_price_path)

# データ前処理
stock_price = standardize_code(stock_price)
df_ohlcv = filter_topix500(stock_price, topix500)

print(f"株価データ行数: {len(df_ohlcv)}")
print(f"日付範囲: {df_ohlcv['Date'].min()} から {df_ohlcv['Date'].max()}")
print(f"ユニークな銘柄数: {df_ohlcv['Code'].nunique()}")

## データ概要の確認

In [ ]:
# データの先頭を確認
df_ohlcv.head()

In [ ]:
# データ型の確認
df_ohlcv.dtypes

In [ ]:
# 日付型への変換
df_ohlcv['Date'] = pd.to_datetime(df_ohlcv['Date'])

# 基本統計量の確認
df_ohlcv.describe()

## 特徴量計算

In [ ]:
# 特徴量計算モジュールのインポート
from src.features.feature_generator import calc_features_and_targets

# ボラティリティ予測用の特徴量計算
df_vol = calc_features_and_targets(df_ohlcv, prediction_type='volatility')

# 日中収益率予測用の特徴量計算
df_ret = calc_features_and_targets(df_ohlcv, prediction_type='intraday_return')

print("ボラティリティ予測用データ形状:", df_vol.shape)
print("日中収益率予測用データ形状:", df_ret.shape)

In [ ]:
# 特徴量と目的変数を確認
vol_features = [col for col in df_vol.columns if 'log_return_lag' in col or 'volatility_5d' in col]
print("ボラティリティ予測用特徴量:")
print(vol_features)
print("\nボラティリティ予測用目的変数: target_vol")

ret_features = [col for col in df_ret.columns if 'log_return_lag' in col or 'volatility_5d' in col]
print("\n日中収益率予測用特徴量:")
print(ret_features)
print("\n日中収益率予測用目的変数: target_return")

## 特徴量の分布の確認

In [ ]:
# 対数収益率の分布
plt.figure(figsize=(12, 6))
plt.hist(df_vol['log_return'].dropna(), bins=100, alpha=0.7)
plt.title('対数収益率の分布')
plt.xlabel('対数収益率')
plt.ylabel('頻度')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 5日間ボラティリティの分布
plt.figure(figsize=(12, 6))
plt.hist(df_vol['volatility_5d'].dropna(), bins=100, alpha=0.7)
plt.title('5日間ボラティリティの分布')
plt.xlabel('ボラティリティ')
plt.ylabel('頻度')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 目的変数の分布
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(df_vol['target_vol'].dropna(), bins=100, alpha=0.7)
plt.title('将来5日間ボラティリティの分布')
plt.xlabel('ボラティリティ')
plt.ylabel('頻度')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(df_ret['target_return'].dropna(), bins=100, alpha=0.7)
plt.title('翌日の日中収益率の分布')
plt.xlabel('収益率')
plt.ylabel('頻度')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 特徴量間の相関関係

In [ ]:
# 特徴量間の相関関係
corr = df_vol[vol_features].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(corr, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('特徴量間の相関係数')
plt.tight_layout()
plt.show()

## 銘柄ごとの特性

In [ ]:
# 銘柄ごとの平均ボラティリティ
code_vol = df_vol.groupby('Code')['volatility_5d'].mean().sort_values(ascending=False)

plt.figure(figsize=(14, 6))
code_vol.head(30).plot(kind='bar')
plt.title('銘柄別平均ボラティリティ（上位30銘柄）')
plt.xlabel('銘柄コード')
plt.ylabel('平均ボラティリティ')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 銘柄ごとの平均日中収益率
code_intraday = df_ret.groupby('Code')['target_return'].mean().sort_values(ascending=False)

plt.figure(figsize=(14, 6))
code_intraday.head(30).plot(kind='bar')
plt.title('銘柄別平均日中収益率（上位30銘柄）')
plt.xlabel('銘柄コード')
plt.ylabel('平均日中収益率')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 時系列の確認

In [ ]:
# 銘柄を1つ選択して時系列を確認
sample_code = topix500[0]  # 最初の銘柄を選択
df_sample = df_vol[df_vol['Code'] == sample_code].sort_values('Date')

# 株価と収益率の時系列
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(df_sample['Date'], df_sample['Close'])
plt.title(f'銘柄 {sample_code} の終値の推移')
plt.ylabel('終値')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(df_sample['Date'], df_sample['log_return'])
plt.title(f'銘柄 {sample_code} の対数収益率の推移')
plt.ylabel('対数収益率')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 3)
plt.plot(df_sample['Date'], df_sample['volatility_5d'])
plt.title(f'銘柄 {sample_code} のボラティリティの推移')
plt.ylabel('5日間ボラティリティ')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## まとめ

このノートブックでは、J-Quants APIから取得した日本株データの基本的な探索を行いました。以下の内容を確認しました：

1. データ取得と前処理
2. 特徴量計算
3. 特徴量の分布と相関関係
4. 銘柄ごとの特性
5. 時系列の確認

次のステップでは、これらの特徴量を使用して、MultiTransformerモデルを構築し、ボラティリティと日中収益率の予測を行います。